# <font color=#0000FF>每10分鐘雲資料重取樣為每15分鐘一筆資料，此外將low為NaN的值用mid來填補，將mid為NaN的值用hig來填補</font>

In [1]:
#將十分鐘一筆資料重取樣為15分鐘一筆。ffill方法用來填補缺失值，確保資料能夠被正確重取樣。
#然後使用isin方法檢查df2(15m)中的日期是否在df(10m)中存在，如果存在則將資料複製到df2中。最後，使用interpolate方法對df2進行線性插值。
import pandas as pd
import numpy as np

# 讀取資料
df = pd.read_csv(f'./cloud_cover/new_fst.v35.csv')

# 將date_time欄位轉換成datetime格式
df['fst_time'] = pd.to_datetime(df['fst_time'])

# 做填充
df['mid'].fillna(df['hig'], inplace=True)
df['low'].fillna(df['mid'], inplace=True)


# 將fst_time欄位設定為索引
df.set_index('fst_time', inplace=True)

# 進行重取樣，將十分鐘一筆資料重取樣為5分鐘一筆
df = df.resample('5T').first().reset_index()

df['lon'].fillna('120.560722', inplace=True)
df['lat'].fillna('24.066583', inplace=True)

df[['low','mid','hig','cloud']] = df[['low','mid','hig','cloud']].interpolate()

# 將dfst_time轉換為datetime格式
df['fst_time'] = pd.to_datetime(df['fst_time'])

# 選擇時間等於15、30、45或00分的資料
df = df[df['fst_time'].dt.minute.isin([15, 30, 45, 0])]

# 刪除date_time欄位
df.drop('date_time', axis=1, inplace=True)

# 將"fst_time"列名更改為"date_time"
df = df.rename(columns={'fst_time': 'date_time'})

# 輸出df2
df.to_csv(f'./cloud_cover/newcloud_10m_to_15m_hig_colud.csv', index=None)

In [2]:
#history_15的csv檔
df_X = pd.read_csv(f'./dataset/solar_汙水廠_newbig_sort(history_15m).csv')
dfXlen = len(df_X)

#篩選時間大於等於2022/2/1小於等於2022/10/31(因為雲資料集只有該區間時間資料)
dfXlen_result = df_X[(df_X['TIME_TO_INTERVAL'] >= '2022-02-01 01:00:00') 
                & (df_X['TIME_TO_INTERVAL'] < '2022-11-01 00:00:00')]

#將原有索引替換為從0開始的新索引
dfXlen_result = dfXlen_result.reset_index(drop=True).reset_index(drop=True)
df = df.reset_index(drop=True)

#合併history_15的csv檔
df_all = pd.concat([dfXlen_result,df],axis=1)

#存成csv
df_all.to_csv(f'./dataset/marge_10m_to_15m_hig_colud_202305.csv', index=None)